In [3]:
!pip install datasets
!pip install tqdm

  Using cached datasets-3.5.0-py3-none-any.whl (491 kB)
  Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
  Using cached aiohttp-3.11.16-cp310-cp310-win_amd64.whl (442 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
  Using cached yarl-1.20.0-cp310-cp310-win_amd64.whl (92 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-win_amd64.whl (51 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl (6.2 kB)
  Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
  Using cached multidict-6.4.3-cp310-cp310-win_amd64.whl (38 kB)
  Using cached idna-3.10-py3-none-


[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Import Dataset
from datasets import load_dataset

data = load_dataset(
    "isek-ai/danbooru-tags-2024",
    split="train",
)

/Users/jiochoi/Projects/anime_character_dataset/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 8603394/8603394 [00:10<00:00, 785024.41 examples/s] 


In [8]:
# Remove original characters, and incomplete data
data = data.filter(lambda x: x["copyright"] is not None and x["character"] is not None and x["general"] is not None and "original" not in x["copyright"])

Filter: 100%|██████████| 8603394/8603394 [02:17<00:00, 62373.35 examples/s]


In [ ]:
# Create raw_characters.json (takes about 20 minutes)
from tqdm import tqdm
import json

characters = {}

for temp in tqdm(data):
    data_characters = temp["character"].split(", ")
    data_copyright = temp["copyright"].split(", ")
    data_general = temp["general"].split(", ")

    # process character data
    for character in data_characters:
        if character not in characters:
            characters[character] = {
                "count": 1,
                "image": "",
                "copyright": {},
                "tags": {}
            }
        else:
            characters[character]["count"] += 1
        
        # process copyright data
        for copyright in data_copyright:
            if copyright not in characters[character]["copyright"]:
                characters[character]["copyright"][copyright] = 1
            else:
                characters[character]["copyright"][copyright] += 1
        
        # process tags data
        if "solo" in data_general:
            for general in data_general:
                if general not in characters[character]["tags"]:
                    characters[character]["tags"][general] = 1
                else:
                    characters[character]["tags"][general] += 1

# Save the characters dictionary to a JSON file
with open("raw_characters.json", "w") as f:
    json.dump(characters, f, indent=4)

  0%|          | 0/7235553 [00:00<?, ?it/s]

100%|██████████| 7235553/7235553 [20:51<00:00, 5781.65it/s]


In [17]:
# Create characters.json (takes about 30 seconds)

from tqdm import tqdm
import json
import hashlib

# Filter out copyrights with image count less than {copyright_threshold}% of the total images
copyright_threshold = 60

# Filter out tags with image count less than {tag_threshold}% of the total images
tag_threshold = 15

# Filter out characters less than {character_threshold} images
character_threshold = 15

# Filter out traits with image count less than {trait_threshold} images
trait_threshold = 5
trait_consideration_threshold = 85

eye_colors = ["aqua eyes", "black eyes", "blue eyes", "brown eyes", "green eyes", "grey eyes", "orange eyes", "purple eyes", "pink eyes", "red eyes", "white eyes", "yellow eyes", "heterochromia"]
hair_colors = ["aqua hair", "black hair", "blonde hair", "blue hair", "brown hair", "green hair", "grey hair", "orange hair", "purple hair", "pink hair", "red hair", "white hair"]
hair_length = ["very short hair", "short hair", "medium hair", "long hair", "very long hair", "absurdly long hair", "big hair", "bald"]
skin_colors = ["dark skin", "pale skin", "black skin", "blue skin", "green skin", "grey skin", "orange skin", "pink skin", "purple skin", "red skin", "silver skin", "white skin", "yellow skin"]
chest_sizes = ["flat chest", "small breasts", "medium breasts", "large breasts", "huge breasts", "gigantic breasts"]
genders = ["1boy", "1girl", "pokemon (creature)"]

with open("raw_characters.json", "r") as f:
    characters = json.load(f)

for character in tqdm(characters):
    count = characters[character]["count"]
    count_tag_max = max(characters[character]["tags"].values()) if len(characters[character]["tags"]) > 0 else 0
    count_copyright_max = max(characters[character]["copyright"].values()) if len(characters[character]["copyright"]) > 0 else 0
    copyright_count = copyright_threshold * count_copyright_max / 100
    tag_count = tag_threshold * count_tag_max / 100
    trait_count = trait_threshold * count_tag_max / 100

    original_tags = characters[character]["tags"].copy()

    # Convert copyrights and tags to list
    characters[character]["copyright"] = {k: v for k, v in characters[character]["copyright"].items() if v >= copyright_count}
    characters[character]["tags"] = {k: v for k, v in original_tags.items() if v >= tag_count}
    
    # Sort the copyrights and tags by count in ascending order in list format
    characters[character]["copyright"] = sorted(characters[character]["copyright"].items(), key=lambda x: x[1], reverse=True)
    characters[character]["tags"] = sorted(characters[character]["tags"].items(), key=lambda x: x[1], reverse=True)

    # Traits
    characters[character]["traits"] = {
        "eye_color": [x for x in original_tags.items() if x[0] in eye_colors and x[1]],
        "hair_color": [x for x in original_tags.items() if x[0] in hair_colors and x[1]],
        "hair_length": [x for x in original_tags.items() if x[0] in hair_length and x[1]],
        "skin_color": [x for x in original_tags.items() if x[0] in skin_colors and x[1] >= trait_count],
        "chest_size": [x for x in original_tags.items() if x[0] in chest_sizes and x[1]],
        "gender": [x for x in original_tags.items() if x[0] in genders and x[1]],
    }

    # Filter out traits with image count less than {trait_consideration_threshold}% of the first trait
    for trait in characters[character]["traits"]:
        if len(characters[character]["traits"][trait]) > 0:
            characters[character]["traits"][trait] = sorted(characters[character]["traits"][trait], key=lambda x: x[1], reverse=True)
            trait_count = characters[character]["traits"][trait][0][1] * trait_consideration_threshold / 100
            characters[character]["traits"][trait] = [x[0] for x in characters[character]["traits"][trait] if x[1] >= trait_count]

    if len(characters[character]["traits"]["skin_color"]) == 0:
        characters[character]["traits"]["skin_color"] = ["normal skin"]

# Convert the dictionary to a list of dictionaries
characters_list = []
for character, data in characters.items():
    if data["count"] < character_threshold:
        continue

    characters_list.append({
        "name": character,
        "hash": hashlib.md5(character.encode()).hexdigest(),
        "count": data["count"],
        "copyright": data["copyright"],
        "tags": data["tags"],
        "traits": data["traits"]
    })

# Sort the list of characters by count in ascending order
characters_list = sorted(characters_list, key=lambda x: x["count"], reverse=True)

characters_min_list = []
for character in characters_list:
    if character["copyright"] == []:
        continue
    if character["tags"] == []:
        continue

    del character["tags"]
    del character["count"]
    del character["copyright"]

    characters_min_list.append(character)

# Save the characters list to a JSON file
with open("characters.json", "w") as f:
    json.dump(characters_list, f, indent=4)
with open("characters_min.json", "w") as f:
    json.dump(characters_min_list, f)

100%|██████████| 235726/235726 [00:19<00:00, 12374.79it/s]


In [3]:
# How many characters in characters.json?
len(characters_list)

52597

In [19]:
# Search test
import json

with open("characters_min.json", "r") as f:
    characters = json.load(f)

def check_trait(character, traits):
    for trait, values in traits.items():
        for value in values:
            if value not in character["traits"][trait]:
                return False
    return True

def search_character(traits):
    results = []
    for character in characters:
        if check_trait(character, traits):
            results.append(character)
    return results[:10]

search_character({
    "eye_color": [],
    "hair_color": ["black hair"],
    "hair_length": ["long hair"],
    "skin_color": [],
    "chest_size": ["small breasts"],
    "gender": ["1girl"]
})


[{'name': 'akemi homura',
  'hash': 'd9e26b9fefc649108e624c33128a5cbb',
  'traits': {'eye_color': ['purple eyes'],
   'hair_color': ['black hair'],
   'hair_length': ['long hair'],
   'skin_color': ['normal skin'],
   'chest_size': ['small breasts'],
   'gender': ['1girl']}},
 {'name': 'akemi homura (magical girl)',
  'hash': '20e0cc6ab47313c307e60981e400e307',
  'traits': {'eye_color': ['purple eyes'],
   'hair_color': ['black hair'],
   'hair_length': ['long hair'],
   'skin_color': ['normal skin'],
   'chest_size': ['small breasts', 'medium breasts'],
   'gender': ['1girl']}},
 {'name': 'akatsuki (kancolle)',
  'hash': '433c1eef7e620aa2cc399cbde1063101',
  'traits': {'eye_color': ['purple eyes'],
   'hair_color': ['purple hair', 'black hair'],
   'hair_length': ['long hair'],
   'skin_color': ['normal skin'],
   'chest_size': ['small breasts'],
   'gender': ['1girl']}},
 {'name': 'yazawa nico',
  'hash': '8df6b1d88945f67a17cb5d4e45015014',
  'traits': {'eye_color': ['red eyes'],
   

In [4]:
# Create copyrights.json

from tqdm import tqdm
import hashlib
import json

with open("characters.json", "r") as f:
    characters = json.load(f)

copyrights = {}

for index in tqdm(range(len(characters))):
    for copyright in characters[index]["copyright"]:
        if copyright[0] not in copyrights:
            copyrights[copyright[0]] = {
                "count": 0,
                "popularity": 0,
                "hash": hashlib.md5(copyright[0].encode()).hexdigest(),
                "characters": []
            }
        
        copyrights[copyright[0]]["count"] += 1
        copyrights[copyright[0]]["characters"].append(characters[index]["hash"])
        copyrights[copyright[0]]["popularity"] = max(copyrights[copyright[0]]["popularity"], characters[index]["count"])
    
# Sort the copyrights by popularity in ascending order in list format
copyrights = sorted(copyrights.items(), key=lambda x: x[1]["popularity"], reverse=True)

with open("copyrights.json", "w") as f:
    json.dump(copyrights, f, indent=4)

100%|██████████| 52597/52597 [00:00<00:00, 424216.41it/s]
